In [1]:
from typing_extensions import TypedDict
from typing import Optional
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage,AIMessage
from langgraph.graph import StateGraph, START, END

In [2]:
model_id = "gemini-2.0-flash-lite-001"
gemini_llm = init_chat_model(model=model_id, model_provider="google_vertexai")

In [4]:
def add(a: int, b: int) -> int:
    """Add a and b

    Args:
        a (int): first int
        b (int): second int

    Returns:
        int: result
    """
    return a + b

In [5]:
gemini_llm.invoke([HumanMessage("What is 2 plus 2")])

AIMessage(content='2 + 2 = 4\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 7, 'candidates_token_count': 8, 'total_token_count': 15, 'prompt_tokens_details': [{'modality': 1, 'token_count': 7}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 8}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.0030223913490772247, 'model_name': 'gemini-2.0-flash-lite-001'}, id='run--370fde31-1898-44b9-af3b-c696a9c22989-0', usage_metadata={'input_tokens': 7, 'output_tokens': 8, 'total_tokens': 15})

In [7]:
# we are binding tools to llm
llm_with_tools = gemini_llm.bind_tools([add])

In [11]:
tool_call = llm_with_tools.invoke([HumanMessage("What is 2 plus 2")])

In [12]:
tool_call.tool_calls

[{'name': 'add',
  'args': {'a': 2.0, 'b': 2.0},
  'id': '1409795f-f65f-43f5-83ac-4fc8953fa4ac',
  'type': 'tool_call'}]

In [13]:
type(tool_call)

langchain_core.messages.ai.AIMessage

In [14]:
tool_call = llm_with_tools.invoke([HumanMessage("What is 2 multiplied by 2")])

In [15]:
tool_call

AIMessage(content="I can only add two numbers.  I can't multiply.\n", additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 24, 'candidates_token_count': 15, 'total_token_count': 39, 'prompt_tokens_details': [{'modality': 1, 'token_count': 24}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 15}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.3100552558898926, 'model_name': 'gemini-2.0-flash-lite-001'}, id='run--0b9b9ba1-12de-4378-b608-ec743a473616-0', usage_metadata={'input_tokens': 24, 'output_tokens': 15, 'total_tokens': 39})

In [16]:
tool_call.tool_calls

[]

### Step 1:  Define the tools

In [61]:
from langchain.agents import tool

In [62]:
@tool
def add(a: int, b: int) -> int:
    """Add a and b

    Args:
        a (int): first int
        b (int): second int

    Returns:
        int: result
    """
    return a + b

In [63]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies  a and b

    Args:
        a (int): first int
        b (int): second int

    Returns:
        int: result
    """
    return a * b

### Step 2: Create ToolNode Wrappers

In [64]:
from langgraph.prebuilt import ToolNode

In [38]:
add_node = ToolNode([add])
multiply_node = ToolNode([multiply])

In [65]:
class State(TypedDict):
    a: int
    b: int

In [66]:
graph_builder = StateGraph(State)

In [74]:
graph_builder.add_node("add", add_node)
graph_builder.add_node("multiply", multiply_node)

In [75]:
graph_builder.add_edge(START, "add")
graph_builder.add_edge("add", END )
#graph_builder.add_edge("multiply", END)

In [76]:
graph = graph_builder.compile()

In [59]:
initial_state = {
    "a": 5,
    "b": 3
}

In [77]:
output = graph.invoke(initial_state)

ValueError: No message found in input